In [1]:
import requests
import yfinance as yf
import pandas as pd
from deep_translator import GoogleTranslator
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# 🌍 Função para traduzir subsetores
def traduzir_subsetor(texto):
    if not texto or texto == "N/A":
        return "N/A"
    try:
        return GoogleTranslator(source='en', target='pt').translate(texto)
    except Exception:
        return texto

# 🔍 Função principal
def coletar_todos_setores_e_subsetores():
    try:
        resposta = requests.get("https://brapi.dev/api/quote/list")
        ativos = resposta.json().get("stocks", [])
    except Exception as e:
        print(f"❌ Erro ao buscar dados da API: {e}")
        return pd.DataFrame()

    registros = []
    for item in ativos:
        ticker = item.get("stock")
        setor_brapi = item.get("sector", "N/A")
        tipo = item.get("type")

        # Ignorar tickers inválidos ou fundos imobiliários
        if not ticker or ticker.endswith("F") or tipo not in {"stock", "fund"}:
            continue

        ticker_formatado = f"{ticker}.SA"
        try:
            info = yf.Ticker(ticker_formatado).info
            if info.get("country") != "Brazil":
                continue

            subsetor_en = info.get("industry", "N/A")
            subsetor_pt = traduzir_subsetor(subsetor_en)

            registros.append({
                "setor_brapi": setor_brapi,
                "subsetor_yfinance": subsetor_en,
                "subsetor_pt": subsetor_pt
            })
        except Exception:
            continue

    df = pd.DataFrame(registros)
    return df.drop_duplicates().reset_index(drop=True)

# 🚀 Executar e exibir
df_setores = coletar_todos_setores_e_subsetores()
print(df_setores)


Exception ignored from cffi callback <function buffer_callback at 0x00000161941A7060>:
Traceback (most recent call last):
  File "C:\Users\Atlas Core\AppData\Roaming\Python\Python313\site-packages\curl_cffi\curl.py", line 100, in buffer_callback
    @ffi.def_extern()
KeyboardInterrupt: 


: 

: 